In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers import ConvLSTM2D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

In [ ]:
f = []
for dirname, _, filenames in os.walk('../input/handgestures/EMG_data_for_gestures-master/EMG_data_for_gestures-master'):
    for filename in filenames:
        f.append(os.path.join(dirname, filename))
f.pop(0)

In [ ]:
df = pd.read_csv(f[0], delimiter='\t')

In [ ]:
df.head()

In [ ]:
Values = []
labels = []
for file in f:
    df = pd.read_csv(file, delimiter='\t')
    Values.append(df.values[:,1:-1])
    labels.append(df.values[:,-1])

In [ ]:
X = []
y = []
for value,label in zip(Values,labels):
    index = 0
    for i,val in enumerate(label):
        if label[index]==val:
            continue
        else:
            X.append(value[index:i])
            y.append(label[index])
            index = i

In [ ]:
X_ = []
y_ = []

for a,b in zip(X,y):
    if b==0.0:
        continue
    else:
        X_.append(a)
        y_.append(b-1)

In [ ]:
curr_max = 0
for val in X_:
    curr_max = max(curr_max,val.shape[0])
curr_max

In [ ]:
Xtrain = pad_sequences(X_)
Ytrain = to_categorical(y_)

In [ ]:
X_train, X_test,Y_train,Y_test = train_test_split(Xtrain, Ytrain, test_size=0.15, random_state=42, stratify = Ytrain)

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(4280,8)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=32)

In [ ]:
def evaluate_model(trainX, trainy, testX, testy, n_steps, n_length):
    # define model
    verbose, epochs, batch_size = 0, 25, 16
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape data into time steps of sub-sequences
    n_steps, n_length = n_steps, n_length
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    # model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    x_pred = model.predict(testX)
    return x_pred,accuracy

def summarize_results(scores):
    # print(scores)
    best, m, s = max(scores), mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
    print("Best: {}".format(best))
    
def run_experiment(trainX, trainy, testX, testy, repeats=1):
    scores = list()
    L = M = [(40,107)]
    
    for data in L:
        # print(data)
        for r in range(repeats):
            data,score = evaluate_model(trainX, trainy, testX, testy, data[0],data[1])
            # return(data)
            score = score * 100.0
            print('>#%d: %.3f' % (r+1, score))
            scores.append(score)
        summarize_results(scores)

In [ ]:
run_experiment(X_train, Y_train, X_test, Y_test)